In [ ]:
!pip install onnxruntime-gpu==1.11 -i http://pypi.douban.com/simple --trusted-host pypi.douban.com

In [ ]:
!pip install netron -i http://pypi.douban.com/simple --trusted-host pypi.douban.com

In [ ]:
!pip install transformers -i http://pypi.douban.com/simple --trusted-host pypi.douban.com

In [1]:
# 我人都傻了, 我说怎么找不到文件, 前面该 notebook 放在 /workspace 目录下, pwd 居然是根目录 /. 真魔幻
!pwd

/workspace/examples


In [8]:
import onnx
import onnxruntime
print(onnxruntime.__version__)
print(onnxruntime.get_device())
print(onnxruntime.get_available_providers())

1.11.0
GPU
['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [3]:
import onnx
import torch
import numpy
from transformers import BertTokenizer
enc = BertTokenizer.from_pretrained('bert-base-uncased')

masked_sentences = ['Paris is the [MASK] of France.', 
                    'The primary [MASK] of the United States is English.', 
                    'A baseball game consists of at least nine [MASK].', 
                    'Topology is a branch of [MASK] concerned with the properties of geometric objects that remain unchanged under continuous transformations.']
pos_masks = [4, 3, 9, 6]

inputs = enc(masked_sentences, return_tensors="np", padding='max_length', max_length=128, truncation=True)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [4]:
from transformers import BertForMaskedLM
origin_model = BertForMaskedLM.from_pretrained("bert-base-uncased", torchscript=True).eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 本地转换模型还是有点报错的, 输出里提到绝对误差超过了 1e-5, 不给导出
!python -m transformers.onnx --model=bert-base-uncased --feature=masked-lm onnx/

In [5]:
for key, val in inputs.items():
    print(key, val.dtype, val.shape)

input_ids int64 (4, 128)
token_type_ids int64 (4, 128)
attention_mask int64 (4, 128)


In [6]:
input_ids = torch.randint(0, 100, (4, 128), device="cpu", dtype=torch.int64)
attention_mask = torch.randint(0, 2, (4, 128), device="cpu", dtype=torch.int64)
token_type_ids = torch.randint(0, 2, (4, 128), device="cpu", dtype=torch.int64)

traced_origin_model = torch.jit.trace(origin_model, [input_ids, attention_mask, token_type_ids])
traced_origin_model(input_ids, attention_mask, token_type_ids)

(tensor([[[-5.3025, -5.4727, -5.5078,  ..., -5.2318, -7.0131,  0.6104],
          [-4.8874, -5.1045, -5.1398,  ..., -4.6891, -6.6637,  0.6894],
          [-4.4334, -4.6704, -4.7000,  ..., -4.4374, -6.2480,  1.1334],
          ...,
          [-4.2189, -4.4351, -4.4214,  ..., -4.1686, -5.9111,  1.2778],
          [-4.0962, -4.3313, -4.3119,  ..., -4.0072, -5.8356,  1.4033],
          [-4.0386, -4.2785, -4.2459,  ..., -3.8926, -5.7575,  1.3704]]],
        grad_fn=<ViewBackward0>),)

In [13]:
!pip install torch==1.13 -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: http://pypi.douban.com/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 890.2 MB 1.6 MB/s eta 0:00:018     |██████████                      | 277.8 MB 379 kB/s eta 0:26:53     |█████████████▉                  | 384.2 MB 388 kB/s eta 0:21:42     |█████████████████▎              | 479.1 MB 290 kB/s eta 0:23:36     |███████████████████████▍        | 651.0 MB 304 kB/s eta 0:13:06
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0a0 requires torch==1.12.0a0+8a1a93a, but you have torch 1.13.0 which is incompatible.
torchtext 0.13.0a0 requires torch==1.12.0a0+8a1a93a, but you have torch 1.13.0 which is incompatible.


In [5]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.13.0+cu117
True


In [3]:
!pip install -U onnx -i http://pypi.douban.com/simple --trusted-host pypi.douban.com

Looking in indexes: http://pypi.douban.com/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 13.5 MB 534 kB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 15.8 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
  Attempting uninstall: onnx
    Found existing installation: onnx 1.11.0
    Uninstalling onnx-1.11.0:
      Successfully uninstalled onnx-1.11.0


使用 `torch.onnx.export` 导出模型的时候需要注意时间, 一般 5 分钟内没导出就需要检查了.

目前尝试的方式是
1. 升级 onnx 版本
2. 升级或降级 torch 版本 (当前降级到 1.11 版本可以导出模型)
3. 还有个最大的原因, 可能是 notebook 造成的, 直接写在 py 代码里运行是可以的

In [7]:
print(origin_model.config.use_cache)
setattr(origin_model.config, "use_cache", False)
print(origin_model.config.use_cache)

True
False


In [5]:
# 导出 ONNX 模型, 宇宙级深坑, 注意检查 jupyter, 直接用 python 代码可以运行的
# 具体和 /opt/conda/lib/python3.8/site-packages/transformers/onnx/convert.py 的 export_pytorch 的函数也没看出有多大的不同

symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
with torch.no_grad():
    traced_origin_model.eval()
    torch.onnx.export(
        traced_origin_model,
        args=(input_ids, attention_mask, token_type_ids),
        f="model_torch.onnx",
        export_params=True,
        verbose=True,
        input_names=["input_ids", "attention_mask", "token_type_ids"],
        output_names=["logits"],
        opset_version=16,
        do_constant_folding=True,
        # 设置动态 shape
        dynamic_axes={
            "input_ids": symbolic_names,
            "attention_mask" : symbolic_names,
            "token_type_ids" : symbolic_names,
        }
    )

In [14]:
onnx.checker.check_model(onnx.load("./onnx/model_torch.onnx"))

In [15]:
# 将 onnx 导出成 tensort 格式
!polygraphy convert ./onnx/model_torch.onnx --convert-to trt -o ./onnx/model_torch.trt --model-type onnx \
    --trt-min-shapes input_ids:[1,128] attention_mask:[1,128] token_type_ids:[1,128] \
    --trt-opt-shapes input_ids:[1,128] attention_mask:[1,128] token_type_ids:[1,128] \
    --trt-max-shapes input_ids:[1,128] attention_mask:[1,128] token_type_ids:[1,128]

[01/11/2023-12:35:23] [TRT] [W] parsers/onnx/onnx2trt_utils.cpp:368: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[01/11/2023-12:35:25] [TRT] [W] Output type must be INT32 for shape outputs
[01/11/2023-12:35:25] [TRT] [W] Output type must be INT32 for shape outputs
[01/11/2023-12:35:25] [TRT] [W] Output type must be INT32 for shape outputs
[01/11/2023-12:35:25] [TRT] [W] Output type must be INT32 for shape outputs
[W] 'colored' module is not installed, will not use colors when logging. To enable colors, please install the 'colored' module: python3 -m pip install colored
[I]     Configuring with profiles: [Profile().add(input_ids, min=[1, 128], opt=[1, 128], max=[1, 128]).add(attention_mask, min=[1, 128], opt=[1, 128], max=[1, 128]).add(token_type_ids, min=[1, 128], opt=[1, 128], max=[1, 128])]
[I] Building engine with configuration:
    Workspace            | 16777216 bytes (16.00 MiB)
    Preci

# 定义三种格式的预测函数

In [85]:
class TorchModel:
    def __init__(self):
        self.enc = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertForMaskedLM.from_pretrained("bert-base-uncased", torchscript=True).eval().cuda()
    
    def predict(self, texts: list, pos_masks: list, return_numpy=True):
        """
        texts 是数组
        pos_masks 是数组,  [MASK] 掩码所在的位置, 注意, 位置需要 +1, 因为前面会填充一个 [CLS] 字符
        'Paris is the [MASK] of France.' 对应的 pos_mask 是 4
        """
        inputs = self.enc(texts, return_tensors="pt", padding='max_length', max_length=128, truncation=True)
        inputs = dict((k, v.cuda()) for k, v in inputs.items())
        with torch.no_grad():
            result = self.model(**inputs)
        if return_numpy:
            result = [x.cpu().numpy() for x in result]
        return result


In [86]:
torch_model = TorchModel()
torch_model.predict(masked_sentences[:1], pos_masks[:1])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[array([[[-6.541635 , -6.5075865, -6.5212164, ..., -5.8960814,
          -5.735184 , -3.8943412],
         [-9.015771 , -9.048779 , -9.0614195, ..., -8.258874 ,
          -8.034796 , -6.17945  ],
         [-8.655979 , -9.090192 , -8.775549 , ..., -7.450444 ,
          -5.341511 , -9.657742 ],
         ...,
         [-8.807926 , -9.008511 , -8.930504 , ..., -8.144446 ,
          -9.269405 , -5.3116097],
         [-8.757712 , -8.873206 , -8.887868 , ..., -8.337829 ,
          -9.368811 , -4.929955 ],
         [-8.782769 , -9.024868 , -8.917733 , ..., -8.314313 ,
          -8.877869 , -6.53116  ]]], dtype=float32)]

In [84]:
class ONNXModel:
    def __init__(self) -> None:
        self.enc = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = onnxruntime.InferenceSession("./onnx/model_torch.onnx", providers=["CUDAExecutionProvider"])
    
    def predict(self, texts: list, pos_masks: list):
        # 输入的数据类型是 np
        inputs = self.enc(texts, return_tensors="np", padding='max_length', max_length=128, truncation=True)
        result = self.model.run(None, input_feed=dict(inputs))
        return result

In [70]:
onnx_model = ONNXModel()
onnx_model.predict(masked_sentences[:1], pos_masks[:1])

[array([[[-6.541909 , -6.5074854, -6.5217304, ..., -5.8963327,
          -5.735438 , -3.8944366],
         [-9.016863 , -9.049933 , -9.063157 , ..., -8.260457 ,
          -8.036411 , -6.180646 ],
         [-8.652298 , -9.086587 , -8.771991 , ..., -7.447172 ,
          -5.337555 , -9.656389 ],
         ...,
         [-8.808946 , -9.009913 , -8.932179 , ..., -8.146702 ,
          -9.2728195, -5.312007 ],
         [-8.759872 , -8.875669 , -8.890579 , ..., -8.340458 ,
          -9.3715725, -4.933445 ],
         [-8.783872 , -9.026353 , -8.919335 , ..., -8.316137 ,
          -8.880703 , -6.534636 ]]], dtype=float32)]

In [28]:
!git clone -b v0.4.0 https://github.com/ELS-RD/transformer-deploy.git

Cloning into 'transformer-deploy'...
remote: Enumerating objects: 2099, done.
remote: Counting objects: 100% (441/441), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 2099 (delta 307), reused 330 (delta 247), pack-reused 1658
Receiving objects: 100% (2099/2099), 33.96 MiB | 82.00 KiB/s, done.
Resolving deltas: 100% (1099/1099), done.
Note: switching to 'ccfeb215fda7ee635068f239b345ab7380f61c6b'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [31]:
import tensorrt as trt
from tensorrt import Logger, Runtime

import os
import sys
sys.path.append("./transformer-deploy/src/")

from transformer_deploy.backends.trt_utils import build_engine, load_engine

In [87]:
class ONNXTensorrtModel:
    def __init__(self) -> None:
        self.enc = BertTokenizer.from_pretrained('bert-base-uncased')
        
        trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
        runtime: Runtime = trt.Runtime(trt_logger)
        profile_index = 0
        self.model = load_engine(
            runtime=runtime,
            engine_file_path="./onnx/model_torch.trt",
            profile_index=profile_index,
        )
    
    def predict(self, texts: list, pos_masks: list, return_numpy=True):
        # 输入的数据类型是 pt
        inputs = self.enc(texts, return_tensors="pt", padding='max_length', max_length=128, truncation=True)
        # 略坑, 需要调整顺序, 所以我不知道这里传入个 dict 有什么意义
        new_inputs = dict()
        for key in ['input_ids', 'attention_mask', 'token_type_ids']:
            new_inputs[key] = inputs[key]
        result = self.model(new_inputs)
        if return_numpy:
            result = [x.cpu().numpy() for x in result]
        return result

In [88]:
trt_model = ONNXTensorrtModel()
trt_model.predict(masked_sentences[:1], pos_masks[:1])

[array([[[-6.541634 , -6.507587 , -6.5212154, ..., -5.896081 ,
          -5.735177 , -3.8943386],
         [-9.015753 , -9.048758 , -9.0613985, ..., -8.25886  ,
          -8.034779 , -6.179438 ],
         [-8.655941 , -9.090158 , -8.775516 , ..., -7.4504123,
          -5.341491 , -9.657685 ],
         ...,
         [-8.80792  , -9.008501 , -8.930499 , ..., -8.144452 ,
          -9.269407 , -5.311605 ],
         [-8.757726 , -8.873215 , -8.887872 , ..., -8.337824 ,
          -9.368818 , -4.929951 ],
         [-8.782761 , -9.024862 , -8.917737 , ..., -8.3143015,
          -8.877874 , -6.531156 ]]], dtype=float32)]

In [78]:
import numpy as np
a = torch_model.predict(masked_sentences[:1], pos_masks[:1])[0]
b = onnx_model.predict(masked_sentences[:1], pos_masks[:1])[0]
c = trt_model.predict(masked_sentences[:1], pos_masks[:1])[0]

print(numpy.allclose(a, b, rtol=1e-03, atol=1e-3))
print(numpy.allclose(a, c, rtol=1e-03, atol=1e-3))

False
True


In [79]:
def print_result(result, masked_sentences, pos_masks):
    most_likely_token_ids = [numpy.argmax(result[i, pos, :]) for i, pos in enumerate(pos_masks)]
    most_likely_probs = [numpy.max(result[i, pos, :]) for i, pos in enumerate(pos_masks)]
    print(most_likely_token_ids, most_likely_probs)
    unmasked_tokens = enc.decode(most_likely_token_ids).split(' ')
    unmasked_sentences = [masked_sentences[i].replace('[MASK]', token) for i, token in enumerate(unmasked_tokens)]
    for sentence in unmasked_sentences:
        print(sentence)

print_result(a, masked_sentences[:1], pos_masks[:1])
print_result(b, masked_sentences[:1], pos_masks[:1])
print_result(c, masked_sentences[:1], pos_masks[:1])

[3007] [18.199736]
Paris is the capital of France.
[3007] [18.201002]
Paris is the capital of France.
[3007] [18.199774]
Paris is the capital of France.


# 简单测试下性能
或许应该将输出归一化成一样的类型, 因为从 cuda 上的 tensor 转换成 numpy 也很耗时的

In [59]:
from functools import partial
import timeit
def timeGraph(call_func, num_loops=50):
    print("Warm up ...")
    for _ in range(20):
        call_func()

    # 等待同步, cuda 默认是异步调用的
    torch.cuda.synchronize()

    print("Start timing ...")
    timings = []
    for i in range(num_loops):
        start_time = timeit.default_timer()
        call_func()
        torch.cuda.synchronize()
        end_time = timeit.default_timer()
        timings.append(end_time - start_time)
        # print("Iteration {}: {:.6f} s".format(i, end_time - start_time))

    return timings


import numpy as np
def printStats(graphName, timings, batch_size):
    times = np.array(timings)
    steps = len(times)
    speeds = batch_size / times
    time_mean = np.mean(times)
    time_med = np.median(times)
    time_99th = np.percentile(times, 99)
    time_std = np.std(times, ddof=0)
    speed_mean = np.mean(speeds)
    speed_med = np.median(speeds)

    msg = ("\n%s =================================\n"
            "batch size=%d, num iterations=%d\n"
            "  Median text batches/second: %.1f, mean: %.1f\n"
            "  Median latency: %.6f, mean: %.6f, 99th_p: %.6f, std_dev: %.6f\n"
            ) % (graphName,
                batch_size, steps,
                speed_med, speed_mean,
                time_med, time_mean, time_99th, time_std)
    print(msg)

In [95]:
timings = timeGraph(partial(torch_model.predict, masked_sentences[:1], pos_masks[:1], False))

printStats("BERT Torch GPU", timings, 1)

Warm up ...
Start timing ...

BERT Torch GPU =================================
batch size=1, num iterations=50
  Median text batches/second: 159.8, mean: 156.1
  Median latency: 0.006259, mean: 0.006544, 99th_p: 0.009934, std_dev: 0.001061



In [96]:
timings = timeGraph(partial(onnx_model.predict, masked_sentences[:1], pos_masks[:1]))

printStats("BERT ONNX GPU", timings, 1)

Warm up ...
Start timing ...

BERT ONNX GPU =================================
batch size=1, num iterations=50
  Median text batches/second: 92.3, mean: 92.7
  Median latency: 0.010836, mean: 0.011059, 99th_p: 0.015159, std_dev: 0.001764



In [98]:
timings = timeGraph(partial(trt_model.predict, masked_sentences[:1], pos_masks[:1], False))

printStats("BERT ONNX TRT GPU", timings, 1)

Warm up ...
Start timing ...

BERT ONNX TRT GPU =================================
batch size=1, num iterations=50
  Median text batches/second: 290.4, mean: 277.7
  Median latency: 0.003444, mean: 0.003658, 99th_p: 0.005146, std_dev: 0.000506



# 总结
从 GPU 复制到 CPU 很耗时.

目前的结果汇总如下

| 模型 | 返回数据类型 | 每秒请求次数 |
| -- | -- | --|
| torch | cuda tensor | 156 |
| torch | numpy | 120 |
| onnx | numpy | 92 |
| onnx_trt | cuda tensor | 277 |
| onnx_trt | numpy | 182 |
